In [1]:
import pandas as pd

### Read csv file and dropped null rows

In [2]:
data=pd.read_csv("data/data_matches.csv")
data=data[['id','season']]
data.head()

,id,season
0,392217,2009
1,1254112,2021
2,829817,2015
3,336022,2007/08
4,501258,2011


In [3]:
data.dropna(inplace=True)

In [4]:
data.isnull().sum()

id        0
season    0
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      905 non-null    int64 
 1   season  905 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.3+ KB


### Read csv file and dropped null rows

In [6]:
updated_ball=pd.read_csv("data/2008_2021_updated_ball.csv")
updated_ball=updated_ball[['id','inning','batsman','batsman_runs']]
updated_ball.head()

,id,inning,batsman,batsman_runs
0,392217,2,M Vijay,1
1,392217,2,ML Hayden,1
2,392217,2,M Vijay,1
3,392217,2,ML Hayden,4
4,392217,2,ML Hayden,0


In [7]:
updated_ball.dropna(inplace=True)

In [8]:
updated_ball.isnull().sum()

id              0
inning          0
batsman         0
batsman_runs    0
dtype: int64

In [9]:
updated_ball.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214949 entries, 0 to 214948
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            214949 non-null  int64 
 1   inning        214949 non-null  int64 
 2   batsman       214949 non-null  object
 3   batsman_runs  214949 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 6.6+ MB


## Overall stats

### Merge above two dataframes 

In [10]:
updated_ball = pd.merge(updated_ball,data,on='id',how='inner')
updated_ball.head(10)

,id,inning,batsman,batsman_runs,season
0,392217,2,M Vijay,1,2009
1,392217,2,ML Hayden,1,2009
2,392217,2,M Vijay,1,2009
3,392217,2,ML Hayden,4,2009
4,392217,2,ML Hayden,0,2009
5,392217,2,ML Hayden,2,2009
6,392217,2,M Vijay,0,2009
7,392217,2,M Vijay,0,2009
8,392217,2,M Vijay,0,2009
9,392217,2,M Vijay,0,2009


In [11]:
updated_ball=updated_ball[updated_ball['season']<='2021']

In [12]:
# Experimenting the total runs 
updated_ball.groupby(
    'batsman').agg(
    {'batsman_runs':sum}).sort_values(
        'batsman_runs',ascending=False).head(10)

,batsman_runs
batsman,
V Kohli,6283
S Dhawan,5784
RG Sharma,5611
SK Raina,5528
DA Warner,5449
AB de Villiers,5162
CH Gayle,4965
MS Dhoni,4746
RV Uthappa,4722


In [13]:
# Experimenting the inning count
temp=updated_ball.groupby(
    ['batsman','id'], as_index=False).count()
temp.groupby(
    'batsman', as_index=False).count().sort_values(
        'batsman_runs',ascending=False)[["batsman","id"]]

# Better to use nunique for count unique values

,batsman,id
396,RG Sharma,207
457,SK Raina,200
527,V Kohli,199
316,MS Dhoni,193
236,KD Karthik,191
...,...,...
51,AS Joseph,1
94,C Nanda,1
381,R Ninan,1
59,Abdur Razzak,1


## Final Result

In [14]:
updated_ball.groupby(
    ['batsman'], as_index=False
    ).agg({
         'id':'nunique',
         'batsman_runs':sum
         }).rename(
             columns={"id":"innings"}
            ).sort_values(
                ['batsman_runs'],ascending=False
                ).reset_index(drop=True).head(10)

,batsman,innings,batsman_runs
0,V Kohli,199,6283
1,S Dhawan,191,5784
2,RG Sharma,207,5611
3,SK Raina,200,5528
4,DA Warner,150,5449
5,AB de Villiers,170,5162
6,CH Gayle,141,4965
7,MS Dhoni,193,4746
8,RV Uthappa,186,4722
9,G Gambhir,151,4217


## Seasonal Stats

In [15]:
seasonal_stats=updated_ball.groupby(
    ['season','batsman'], as_index=False).agg(
        {'id':'nunique','batsman_runs':sum}).rename(
             columns={"id":"innings"})

In [16]:
seasonal_stats.head(10)

,season,batsman,innings,batsman_runs
0,2007/08,A Chopra,5,42
1,2007/08,A Kumble,4,13
2,2007/08,A Mishra,3,37
3,2007/08,A Mukund,1,0
4,2007/08,A Nehra,3,3
5,2007/08,A Symonds,3,161
6,2007/08,AA Noffke,1,9
7,2007/08,AB Agarkar,6,54
8,2007/08,AB Dinda,2,2
9,2007/08,AB de Villiers,6,95


In [17]:
seasonal_stats[seasonal_stats["season"]=="2016"].sort_values(
            ['batsman_runs'],ascending=False).reset_index(drop=True).head(10)

,season,batsman,innings,batsman_runs
0,2016,V Kohli,16,973
1,2016,DA Warner,17,848
2,2016,AB de Villiers,16,687
3,2016,S Dhawan,17,501
4,2016,G Gambhir,15,501
5,2016,RG Sharma,14,489
6,2016,AM Rahane,14,480
7,2016,M Vijay,14,453
8,2016,Q de Kock,13,445
9,2016,SK Raina,15,399


In [24]:
def overall_stats(updated_ball, batsman):
    result = updated_ball.groupby(
    ['batsman'], as_index=False
    ).agg({
         'id':'nunique',
         'batsman_runs':sum
         }).rename(
             columns={"id":"innings"}
            ).sort_values(
                ['batsman_runs'],ascending=False
                ).reset_index(drop=True)
    
    batsman_result =  result[result["batsman"]== batsman]
    return result,batsman_result

In [25]:
overall_stats(updated_ball, "V Kohli")

(            batsman  innings  batsman_runs
 0           V Kohli      199          6283
 1          S Dhawan      191          5784
 2         RG Sharma      207          5611
 3          SK Raina      200          5528
 4         DA Warner      150          5449
 ..              ...      ...           ...
 556         C Nanda        1             0
 557     Sunny Gupta        1             0
 558      RR Bhatkal        1             0
 559      Anuj Rawat        1             0
 560  V Pratap Singh        1             0
 
 [561 rows x 3 columns],
    batsman  innings  batsman_runs
 0  V Kohli      199          6283)

In [27]:
def seasonal_stats(updated_ball,year):
    result=updated_ball.groupby(
    ['season','batsman'], as_index=False).agg(
        {'id':'nunique','batsman_runs':sum}).rename(
             columns={"id":"innings"})
    
    yearwise_result = result[result["season"]==year].sort_values(
            ['batsman_runs'],ascending=False).reset_index(drop=True)
    
    return result,yearwise_result

In [28]:
seasonal_stats(updated_ball,"2016")

(       season            batsman  innings  batsman_runs
 0     2007/08           A Chopra        5            42
 1     2007/08           A Kumble        4            13
 2     2007/08           A Mishra        3            37
 3     2007/08           A Mukund        1             0
 4     2007/08            A Nehra        3             3
 ...       ...                ...      ...           ...
 2090     2021        Virat Singh        2            15
 2091     2021            WP Saha        9           131
 2092     2021  Washington Sundar        4            31
 2093     2021        YBK Jaiswal       10           249
 2094     2021          YS Chahal        2            10
 
 [2095 rows x 4 columns],
     season         batsman  innings  batsman_runs
 0     2016         V Kohli       16           973
 1     2016       DA Warner       17           848
 2     2016  AB de Villiers       16           687
 3     2016        S Dhawan       17           501
 4     2016       G Gambhir      